# Recommender Systems 2021/22

### Practice - FunkSVD and SVD++ implemented with Python and Cython

FunkSVD is one of the simplest and most known matrix factorization models for rating prediction. It was proposed by Simon Funk in a now famous post on his website (with a cow grazing around Auckland): https://sifter.org/~simon/journal/20061211.html

SVD++ is an extension of FunkSVD that learns also the biases: global, user and item.

In [1]:
import time

import numpy as np

In [3]:
import scipy.sparse as sps

from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from challenge.utils.functions import read_data, evaluate_algorithm, generate_submission_csv


data_file_path = '../challenge/input_files/data_train.csv'
users_file_path = '../challenge/input_files/data_target_users_test.csv'
URM_all_dataframe, users_list = read_data(data_file_path, users_file_path)

URM_all = sps.coo_matrix(
    (URM_all_dataframe['Data'].values, (URM_all_dataframe['UserID'].values, URM_all_dataframe['ItemID'].values)))
URM_all = URM_all.tocsr()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)

In [4]:
URM_train

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>

### What do we need for FunkSVD and SVD++?

* Loss function
* User factor and Item factor matrices
* Computing prediction
* Update rule
* Training loop and some patience


In [5]:
n_users, n_items = URM_train.shape

#### The two methods are based on two latent factor matrices $ W \in R^{U \times E} , V \in R^{E \times I}$ with E the embedding size, and biases $ \mu \in R , b_u \in R^{U}, b_i \in R^{I}$

#### How to compute the predictions
FunkSVD: $ \hat{r}_{ui} = \sum_{j=0}^{E} W_{uj}H_{ji}$

SVD++: $ \hat{r}_{ui} = \mu + b_u + b_i + \sum_{j=0}^{E} W_{uj}H_{ji}$


#### The loss function we are interested in minimizing are

$L_{FunkSVD} = |R - WH|_F + \alpha|W|_2 + \beta|H|_2$

$L_{SVD++} = |R - (\mu + b_u + b_i + WH)|_F + \alpha|W|_2 + \beta|H|_2 + \gamma (\mu + b_u + b_i) $

Notice that in this case the loss is the Frobenius norm, not the 2 norm, hence we want to minimize the prediction error only for the existing ratings not the missing ones. In practice this means one samples only among the observed interactions.

While this approach works well for rating prediction, it does not for ranking. A model must be trained also on negative data if it expected to learn how to distinguish between positive and negative data. A good strategy is to randomly sample unobserved interactions during training assigning them a rating of 0.

#### Gradients

$\frac{\partial}{\partial W} L = -2(R - WH)H + 2\alpha W $

$\frac{\partial}{\partial H} L = -2(R - WH)W + 2\alpha H $

$\frac{\partial}{\partial \mu} L = -2(R - WH) + 2\gamma \mu $

$\frac{\partial}{\partial b_u} L = -2(R - WH) + 2\gamma b_u $

$\frac{\partial}{\partial b_i} L = -2(R - WH) + 2\gamma b_i $

#### The update is going to be (we can remove the coefficients)
$ W = W - \frac{\partial}{\partial W}$, or 

$ W = W + l((R - WH)H - \alpha W)$, with $l$ the learning rate

... and similarly for the other parameters: $H$, $\mu$, $b_u$ and $b_i$.

## Step 1: We create the dense latent factor matrices
### In a MF model you have two matrices, one with a row per user and the other with a column per item. The other dimension, columns for the first one and rows for the second one is called latent factors

In [6]:
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

In [7]:
user_factors

array([[0.55497448, 0.74147952, 0.95997093, ..., 0.52470196, 0.30168058,
        0.72492914],
       [0.44407834, 0.63215542, 0.99257338, ..., 0.5204189 , 0.1150253 ,
        0.06760315],
       [0.45558917, 0.10335099, 0.67634476, ..., 0.42545073, 0.5659515 ,
        0.92216743],
       ...,
       [0.44096602, 0.42592716, 0.02274097, ..., 0.69456287, 0.67570268,
        0.50807566],
       [0.6638893 , 0.78363141, 0.41890678, ..., 0.4130172 , 0.42374994,
        0.25706849],
       [0.25018272, 0.10236679, 0.21494552, ..., 0.21454257, 0.73481667,
        0.23705308]])

In [8]:
item_factors

array([[0.91875095, 0.63265318, 0.24853968, ..., 0.32276046, 0.01621912,
        0.70902597],
       [0.34848675, 0.12744108, 0.48779798, ..., 0.53065094, 0.02945919,
        0.51063867],
       [0.73984351, 0.39227488, 0.61184284, ..., 0.19498732, 0.53586606,
        0.91884103],
       ...,
       [0.94597969, 0.13433809, 0.25462766, ..., 0.22941094, 0.72050609,
        0.91202238],
       [0.15138569, 0.12825743, 0.93197362, ..., 0.45444849, 0.44825643,
        0.69646829],
       [0.51231646, 0.66190746, 0.05499428, ..., 0.81103923, 0.34508031,
        0.5805828 ]])

## Step 2: We sample an interaction and compute the prediction of the current FunkSVD model

In [9]:
URM_train_coo = URM_train.tocoo()

sample_index = np.random.randint(URM_train_coo.nnz)
sample_index

349082

In [10]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
rating = URM_train_coo.data[sample_index]

(user_id, item_id, rating)

(11850, 2826, 1.0)

In [11]:
predicted_rating = np.dot(user_factors[user_id, :], item_factors[item_id, :])
predicted_rating

2.5350327533562833

#### The first predicted rating is a random prediction, essentially

### Step 3: We compute the prediction error and update the latent factor matrices

In [12]:
prediction_error = rating - predicted_rating
prediction_error

-1.5350327533562833

### The error is positive, so we need to increase the prediction our model computes. Meaning, we have to increase the values latent factor matrices

### Which latent factors we modify? All the factors of the item and user we used

In [13]:
# Copy original value to avoid messing up the updates
H_i = item_factors[item_id, :]
W_u = user_factors[user_id, :]

In [14]:
H_i

array([0.65614443, 0.70285965, 0.74044875, 0.51979078, 0.47240703,
       0.53513915, 0.17941934, 0.33524476, 0.61605554, 0.63896619])

In [15]:
W_u

array([0.34430595, 0.15376681, 0.51274351, 0.57062761, 0.97609055,
       0.70085867, 0.01271441, 0.52239013, 0.11718933, 0.68704845])

#### Apply the update rule

In [16]:
learning_rate = 1e-4
regularization = 1e-5

In [17]:
user_factors_update = prediction_error * H_i - regularization * W_u
user_factors_update

array([-1.00720663, -1.07891412, -1.13661821, -0.79790158, -0.72517003,
       -0.82146313, -0.27541469, -0.5146169 , -0.94566661, -0.9808409 ])

In [18]:
item_factors_update = prediction_error * W_u - regularization * H_i
item_factors_update

array([-0.52852747, -0.23604413, -0.78708548, -0.87593727, -1.49833568,
       -1.07584636, -0.01951883, -0.80188931, -0.17989562, -1.05464826])

In [19]:
user_factors[user_id, :] += learning_rate * user_factors_update
item_factors[item_id, :] += learning_rate * item_factors_update

### Let's check what the new prediction for the same user-item interaction would be

In [20]:
predicted_rating = np.dot(user_factors[user_id, :], item_factors[item_id, :])
predicted_rating

2.5340936137175905

### The value is higher than before, we are moving in the right direction

### And now? Sample another interaction and repeat... a lot of times

### WARNING: Initialization must be done with random non-zero values ... otherwise

In [21]:
user_factors = np.zeros((n_users, num_factors))
item_factors = np.zeros((n_items, num_factors))

predicted_rating = np.dot(user_factors[user_id, :], item_factors[item_id, :])

print("Prediction is {:.2f}".format(predicted_rating))

prediction_error = rating - predicted_rating

print("Prediction error is {:.2f}".format(prediction_error))


Prediction is 0.00
Prediction error is 1.00


In [22]:
H_i = item_factors[item_id, :]
W_u = user_factors[user_id, :]

user_factors[user_id, :] += learning_rate * (prediction_error * H_i - regularization * W_u)
item_factors[item_id, :] += learning_rate * (prediction_error * W_u - regularization * H_i)

In [23]:
predicted_rating = np.dot(user_factors[user_id, :], item_factors[item_id, :])

print("Prediction after the update is {:.2f}".format(predicted_rating))
print("Prediction error is {:.2f}".format(rating - predicted_rating))

Prediction after the update is 0.00
Prediction error is 1.00


### Since the matrices are multiplied, if we initialize one of them as zero, the updates will always be zero and the model will not be able to learn.

### Let's put all together in a training loop.

In [24]:
URM_train_coo = URM_train.tocoo()

num_factors = 10
learning_rate = 1e-4
regularization = 1e-5

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

loss = 0.0
start_time = time.time()

for sample_num in range(1000000):

    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = np.dot(user_factors[user_id, :], item_factors[item_id, :])

    # Compute prediction error, or gradient
    prediction_error = rating - predicted_rating
    loss += prediction_error ** 2

    # Copy original value to avoid messing up the updates
    H_i = item_factors[item_id, :]
    W_u = user_factors[user_id, :]

    user_factors_update = prediction_error * H_i - regularization * W_u
    item_factors_update = prediction_error * W_u - regularization * H_i

    user_factors[user_id, :] += learning_rate * user_factors_update
    item_factors[item_id, :] += learning_rate * item_factors_update

    # Print some stats
    if (sample_num + 1) % 100000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num / elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num + 1,
                                                                                                 elapsed_time,
                                                                                                 loss / sample_num,
                                                                                                 samples_per_second))

Iteration 100000 in 1.93 seconds, loss is 2.78. Samples per second 51895.99
Iteration 200000 in 3.85 seconds, loss is 2.73. Samples per second 51970.47
Iteration 300000 in 5.78 seconds, loss is 2.69. Samples per second 51899.16
Iteration 400000 in 7.77 seconds, loss is 2.65. Samples per second 51497.08
Iteration 500000 in 9.72 seconds, loss is 2.62. Samples per second 51446.64
Iteration 600000 in 11.67 seconds, loss is 2.58. Samples per second 51409.62
Iteration 700000 in 13.63 seconds, loss is 2.55. Samples per second 51339.33
Iteration 800000 in 15.60 seconds, loss is 2.51. Samples per second 51295.25
Iteration 900000 in 17.54 seconds, loss is 2.48. Samples per second 51311.99
Iteration 1000000 in 19.47 seconds, loss is 2.45. Samples per second 51350.43


### What do we see? The loss generally goes down but may oscillate a bit.
### How long do we train such a model?

* An epoch: a complete loop over all the train data
* Usually you train for multiple epochs. Depending on the algorithm and data 10s or 100s of epochs.

In [25]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds,
                                                                                                    estimated_seconds / 60))

Estimated time with the previous training speed is 1557.92 seconds, or 25.97 minutes


### This model is relatively quick

### Let's see what we can do with Cython
### First step, just compile it. We do not have the data at compile time, so we put the loop in a function

In [29]:
%load_ext Cython
import time

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [34]:
%%cython

import time
import numpy as np

def do_some_training(URM_train):
    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape

    num_factors = 10
    learning_rate = 1e-4
    regularization = 1e-5

    user_factors = np.random.random((n_users, num_factors))
    item_factors = np.random.random((n_items, num_factors))

    loss = 0.0
    start_time = time.time()

    for sample_num in range(1000000):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = np.dot(user_factors[user_id, :], item_factors[item_id, :])

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error ** 2

        # Copy original value to avoid messing up the updates
        H_i = item_factors[item_id, :]
        W_u = user_factors[user_id, :]

        user_factors_update = prediction_error * H_i - regularization * W_u
        item_factors_update = prediction_error * W_u - regularization * H_i

        user_factors[user_id, :] += learning_rate * user_factors_update
        item_factors[item_id, :] += learning_rate * item_factors_update

        # Print some stats
        if (sample_num + 1) % 100000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num / elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num + 1,
                                                                                                     elapsed_time,
                                                                                                     loss / sample_num,
                                                                                                     samples_per_second))

    return loss, samples_per_second

Content of stderr:
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_90e8558e1b81eb50f1573fb5776199a8b5dd6ba3.c:5228:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_90e8558e1b81eb50f1573fb5776199a8b5dd6ba3.c:531:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallthrough))
                                 ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_90e8558e1b81eb50f1573fb5776199a8b5dd6ba3.c:5239:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_90e8558e1b81eb50f1573fb5776199a8b5dd6ba3.c:531:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallth

In [35]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 100000 in 2.38 seconds, loss is 2.69. Samples per second 41996.78
Iteration 200000 in 4.16 seconds, loss is 2.66. Samples per second 48067.78
Iteration 300000 in 5.90 seconds, loss is 2.62. Samples per second 50878.26
Iteration 400000 in 7.67 seconds, loss is 2.58. Samples per second 52139.84
Iteration 500000 in 9.44 seconds, loss is 2.54. Samples per second 52964.06
Iteration 600000 in 11.21 seconds, loss is 2.51. Samples per second 53505.40
Iteration 700000 in 12.97 seconds, loss is 2.48. Samples per second 53958.37
Iteration 800000 in 14.77 seconds, loss is 2.45. Samples per second 54168.71
Iteration 900000 in 16.55 seconds, loss is 2.41. Samples per second 54390.19
Iteration 1000000 in 18.41 seconds, loss is 2.39. Samples per second 54325.18


In [36]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds,
                                                                                                    estimated_seconds / 60))

Estimated time with the previous training speed is 1472.61 seconds, or 24.54 minutes


### The compiler is just porting in C all operations that the python interpreter would have to perform, dynamic tiping included. Have a look at the html reports in the Cython_examples folder

### Now try to add some types: If you use a variable only as a C object, use primitive tipes

* cdef int namevar
* cdef double namevar
* cdef float namevar
* cdef double[:] singledimensionarray
* cdef double[:,:] bidimensionalmatrix

### Some operations are still done with sparse matrices, those cannot be correctly optimized because the compiler does not know how what is the type of the data.

### To address this, we create typed arrays in which we put the URM_train data
For example, this operation: user_id = URM_train_coo.row[sample_index]

Becomes:
cdef int user_id
cdef int[:] URM_train_coo_row = URM_train_coo.row
user_id = URM_train_coo_row[sample_index]

### We can also skip the creation of the items_in_user_profile array and replace the np.random call with the faster native C function rand()


### We now use types for all main variables


In [48]:
%%cython

import time
import numpy as np

from libc.stdlib cimport rand, srand, RAND_MAX


def do_some_training(URM_train):
    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape
    cdef int n_interactions = URM_train.nnz

    cdef int sample_num, sample_index, user_id, item_id, factor_index
    cdef double rating, predicted_rating, prediction_error

    cdef int num_factors = 10
    cdef double learning_rate = 1e-4
    cdef double regularization = 1e-5

    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data

    cdef double[:, :] user_factors = np.random.random((n_users, num_factors))
    cdef double[:, :] item_factors = np.random.random((n_items, num_factors))
    cdef double H_i, W_u
    cdef double item_factors_update, user_factors_update

    cdef double loss = 0.0
    cdef long start_time = time.time()
    
    cdef double samples_per_second = 0.0

    for sample_num in range(URM_train.nnz):

        # Randomly pick sample
        sample_index = rand() % n_interactions

        user_id = URM_train_coo_row[sample_index]
        item_id = URM_train_coo_col[sample_index]
        rating = URM_train_coo_data[sample_index]

        # Compute prediction
        predicted_rating = 0.0

        for factor_index in range(num_factors):
            predicted_rating += user_factors[user_id, factor_index] * item_factors[item_id, factor_index]

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error ** 2

        # Copy original value to avoid messing up the updates
        for factor_index in range(num_factors):
            H_i = item_factors[item_id, factor_index]
            W_u = user_factors[user_id, factor_index]

            user_factors_update = prediction_error * H_i - regularization * W_u
            item_factors_update = prediction_error * W_u - regularization * H_i

            user_factors[user_id, factor_index] += learning_rate * user_factors_update
            item_factors[item_id, factor_index] += learning_rate * item_factors_update

            # Print some stats
        if (sample_num + 1) % 100000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num / elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(
                sample_num + 1, elapsed_time,loss / sample_num, samples_per_second))

    return loss, samples_per_second

Content of stderr:
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_351dc821f31a30486329e7809111d89dbc4fb119.c:22973:26: warning: code will never be executed [-Wunreachable-code]
                module = PyImport_ImportModuleLevelObject(
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_351dc821f31a30486329e7809111d89dbc4fb119.c:22973:26: warning: code will never be executed [-Wunreachable-code]
                module = PyImport_ImportModuleLevelObject(
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.

In [49]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 100000 in 0.77 seconds, loss is 2.69. Samples per second 130577.51
Iteration 200000 in 0.79 seconds, loss is 2.66. Samples per second 251703.07
Iteration 300000 in 0.82 seconds, loss is 2.62. Samples per second 364330.25


In [50]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds,
                                                                                                    estimated_seconds / 60))

Estimated time with the previous training speed is 219.58 seconds, or 3.66 minutes


### Works nicely, let's put an additional for loop to do multiple epochs

In [79]:
%%cython
import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def train_multiple_epochs(URM_train, n_epochs):
    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape
    
    cdef int n_interactions = URM_train.nnz

    cdef int sample_num, sample_index, user_id, item_id, factor_index
    cdef double rating, predicted_rating, prediction_error

    cdef int num_factors = 10
    cdef double learning_rate = 1e-2
    cdef double regularization = 1e-3

    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data

    cdef double[:, :] user_factors = np.random.random((n_users, num_factors))
    cdef double[:, :] item_factors = np.random.random((n_items, num_factors))
    cdef double H_i, W_u
    cdef double item_factors_update, user_factors_update

    cdef double loss = 0.0
    cdef long start_time = time.time()
    
    cdef double samples_per_second = 0.0

    for n_epoch in range(n_epochs):

        loss = 0.0
        start_time = time.time()

        for sample_num in range(URM_train.nnz):

            # Randomly pick sample
            sample_index = rand() % n_interactions

            user_id = URM_train_coo_row[sample_index]
            item_id = URM_train_coo_col[sample_index]
            rating = URM_train_coo_data[sample_index]

            # Compute prediction
            predicted_rating = 0.0

            for factor_index in range(num_factors):
                predicted_rating += user_factors[user_id, factor_index] * item_factors[item_id, factor_index]

            # Compute prediction error, or gradient
            prediction_error = rating - predicted_rating
            loss += prediction_error ** 2

            # Copy original value to avoid messing up the updates
            for factor_index in range(num_factors):
                H_i = item_factors[item_id, factor_index]
                W_u = user_factors[user_id, factor_index]

                user_factors_update = prediction_error * H_i - regularization * W_u
                item_factors_update = prediction_error * W_u - regularization * H_i

                user_factors[user_id, factor_index] += learning_rate * user_factors_update
                item_factors[item_id, factor_index] += learning_rate * item_factors_update

        elapsed_time = time.time() - start_time
        samples_per_second = sample_num / elapsed_time

        print("Epoch {} complete in in {:.2f} seconds, loss is {:.3E}. Samples per second {:.2f}".format(n_epoch + 1,
                                                                                                         time.time() - start_time,
                                                                                                         loss / sample_num,
                                                                                                         samples_per_second))

    return np.array(user_factors), np.array(item_factors), loss, samples_per_second

Content of stderr:
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_c4d1772a7b67729ab47289e0bd899dc0c9086516.c:23241:26: warning: code will never be executed [-Wunreachable-code]
                module = PyImport_ImportModuleLevelObject(
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_c4d1772a7b67729ab47289e0bd899dc0c9086516.c:23756:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_c4d1772a7b67729ab47289e0bd899dc0c9086516.c:540:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallthrough))
                                 ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_c4d1772a7b67729ab47289e0bd899dc0c9086516.c:23767:21: warning: fallthrough annotation in unreachable code [-Wunreachable-co

In [89]:
n_items = URM_train.shape[1]

user_factors, item_factors, loss, samples_per_second = train_multiple_epochs(URM_train,10000)

Epoch 1 complete in in 0.35 seconds, loss is 1.850E+00. Samples per second 1081445.79
Epoch 2 complete in in 0.41 seconds, loss is 1.072E+00. Samples per second 929243.58
Epoch 3 complete in in 0.47 seconds, loss is 7.695E-01. Samples per second 817799.92
Epoch 4 complete in in 0.54 seconds, loss is 6.029E-01. Samples per second 707028.01
Epoch 5 complete in in 0.60 seconds, loss is 5.014E-01. Samples per second 633951.85
Epoch 6 complete in in 0.67 seconds, loss is 4.274E-01. Samples per second 571916.13
Epoch 7 complete in in 0.73 seconds, loss is 3.774E-01. Samples per second 525366.90
Epoch 8 complete in in 0.80 seconds, loss is 3.368E-01. Samples per second 480246.33
Epoch 9 complete in in 0.87 seconds, loss is 3.045E-01. Samples per second 442267.67
Epoch 10 complete in in 0.94 seconds, loss is 2.788E-01. Samples per second 406507.14
Epoch 11 complete in in 1.05 seconds, loss is 2.563E-01. Samples per second 365371.02
Epoch 12 complete in in 0.25 seconds, loss is 2.387E-01. Sampl

### From 20 minutes of training time to a few seconds...

In [90]:
user_factors

array([[ 0.14872001,  0.70707497,  0.85412587, ...,  0.46381152,
         0.643044  ,  0.86745916],
       [ 0.28520544,  0.23846745,  0.15257217, ...,  0.23225976,
         0.21051827,  0.32838258],
       [ 0.07387256,  0.54291947,  0.19203001, ...,  0.11157628,
         0.1641683 ,  0.4239286 ],
       ...,
       [ 0.74547198,  0.44806944,  0.59785718, ..., -0.11277861,
         0.35988401,  0.29863731],
       [ 0.11818504,  0.24684614,  0.63982747, ...,  0.65344805,
         0.08817833,  0.11359554],
       [ 0.25588062,  0.24536337,  0.25306386, ...,  0.25635248,
         0.25708048,  0.27177925]])

In [91]:
item_factors

array([[ 2.79395614e-01,  3.05289619e-01,  6.08441349e-01, ...,
         2.54151906e-01,  7.26433120e-01,  8.83010463e-01],
       [ 3.71417842e-01,  3.71493707e-01,  3.68484964e-01, ...,
         3.87275558e-01,  3.80158565e-01,  3.96420600e-01],
       [ 3.71756380e-01,  3.70610028e-01,  3.66554101e-01, ...,
         3.87040038e-01,  3.81388309e-01,  3.96795325e-01],
       ...,
       [-3.81035061e-04,  8.59252945e-01,  6.53320171e-01, ...,
        -3.92261277e-02,  3.69527817e-01,  7.72102802e-01],
       [ 4.52722904e-01,  4.21951934e-01,  6.03225208e-01, ...,
        -2.21070016e-02,  1.85405542e-01,  5.68741247e-01],
       [ 4.02285477e-01, -8.07056530e-02,  7.12846995e-01, ...,
         3.30139076e-01,  2.02796028e-01,  8.59108366e-01]])

In [94]:
def recommend(user_id, user_factors, item_factors, URM_train, at=10):
    # Compute user predicted ratings
    user_ratings = user_factors[user_id - 1, :].dot(item_factors.T)

    # Remove the seen items
    user_ratings = user_ratings.squeeze()
    item_ids = np.arange(0, user_ratings.shape[0], dtype=np.int)
    item_ids = np.delete(item_ids, URM_train[user_id].indices)
    user_ratings = np.delete(user_ratings, URM_train[user_id].indices)

    # Sort the indices
    ranking = np.argsort(user_ratings)[::-1]

    return item_ids[ranking[:at]]

In [95]:
user_id = 59
recommendations = recommend(user_id, user_factors, item_factors, URM_train, at=10)
recommendations

/var/folders/45/5q6zskw54k5crwq2ngs9b_mh0000gn/T/ipykernel_1437/1935753080.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  item_ids = np.arange(0, user_ratings.shape[0], dtype=np.int)


array([14193, 19417, 16177, 22182, 17640, 21822, 19902, 13535,  7355,
       20879])

In [96]:
recommendations = []
for user_id in users_list:
    recommended_items = recommend(user_id, user_factors, item_factors, URM_train, at=10)
    recommendation = {"user_id": user_id, "item_list": recommended_items}
    print("User {} is recommended {}".format(user_id, recommended_items))
    recommendations.append(recommendation)

/var/folders/45/5q6zskw54k5crwq2ngs9b_mh0000gn/T/ipykernel_1437/1935753080.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  item_ids = np.arange(0, user_ratings.shape[0], dtype=np.int)


User 1 is recommended [19417 19902 16177 14193 20879  7355 17640 13535 21822 10693]
User 2 is recommended [14193 17640 19417 21383 16177 22182  7355 12074 21822 13535]
User 3 is recommended [ 7355 17640 22182 14193 12074 20747 16214 10693 19902 19417]
User 4 is recommended [22182 12644 17640 14193 16419 16214 21822  7355 21586 18382]
User 5 is recommended [21822 13535 15393 21586 17640 22182 19268 16177  9399  8970]
User 6 is recommended [14193 19417 22182 16177 17640 21822 19902 13535  7355 21383]
User 8 is recommended [16177 15469 13535 19902 19765 17640 22236 16214 15564 14320]
User 9 is recommended [17640 22182  7355 16214 13535 14193 12074 17001 20747 19902]
User 10 is recommended [16177 17640 21822  8970 15055 15393 21586 13535 22182 18689]
User 11 is recommended [14193 19417 16177 22182 21822 19902 13535 20879 17640 21383]
User 12 is recommended [13535 16177 20879 19268 19417 19495 14193  9155  9399 19635]
User 13 is recommended [14193 19417 16177 22182 21822 17640 19902 13535  

In [97]:
def mapping(recommended_items, relevant_items):
    precision_at_k = 0.0
    num_correct = 0

    for i, item in enumerate(recommended_items):
        if item in relevant_items:
            num_correct += 1
            precision_at_k += num_correct / (i + 1)

    if not relevant_items:
        return 0.0

    return precision_at_k / len(relevant_items)

def evaluate_algorithm(URM_test, user_factors, item_factors, users_list, at=10):
    cumulative_MAP = 0.0
    num_eval = 0

    for user_id in users_list:

        relevant_items = set(URM_test[user_id].indices)

        if len(relevant_items) > 0:

            recommended_items = recommend(user_id, user_factors, item_factors, URM_train, at=at)
            num_eval += 1

            cumulative_MAP += mapping(recommended_items, relevant_items)

    cumulative_MAP /= num_eval

    print("Recommender performance is: MAP = {:.4f}".format(cumulative_MAP))
    return cumulative_MAP

In [98]:
recommendations

[{'user_id': 1,
  'item_list': array([19417, 19902, 16177, 14193, 20879,  7355, 17640, 13535, 21822,
         10693])},
 {'user_id': 2,
  'item_list': array([14193, 17640, 19417, 21383, 16177, 22182,  7355, 12074, 21822,
         13535])},
 {'user_id': 3,
  'item_list': array([ 7355, 17640, 22182, 14193, 12074, 20747, 16214, 10693, 19902,
         19417])},
 {'user_id': 4,
  'item_list': array([22182, 12644, 17640, 14193, 16419, 16214, 21822,  7355, 21586,
         18382])},
 {'user_id': 5,
  'item_list': array([21822, 13535, 15393, 21586, 17640, 22182, 19268, 16177,  9399,
          8970])},
 {'user_id': 6,
  'item_list': array([14193, 19417, 22182, 16177, 17640, 21822, 19902, 13535,  7355,
         21383])},
 {'user_id': 8,
  'item_list': array([16177, 15469, 13535, 19902, 19765, 17640, 22236, 16214, 15564,
         14320])},
 {'user_id': 9,
  'item_list': array([17640, 22182,  7355, 16214, 13535, 14193, 12074, 17001, 20747,
         19902])},
 {'user_id': 10,
  'item_list': array([1

In [99]:
evaluate_algorithm(URM_test, user_factors, item_factors, users_list, at=10)

/var/folders/45/5q6zskw54k5crwq2ngs9b_mh0000gn/T/ipykernel_1437/1935753080.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  item_ids = np.arange(0, user_ratings.shape[0], dtype=np.int)


Recommender performance is: MAP = 0.0000


2.2902826874483308e-05

In [100]:
generate_submission_csv("submission.csv", recommendations)